In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import time

In [2]:
chrome_options = webdriver.ChromeOptions()

In [3]:
chrome_options.binary_location = 'C:/chromedriver/chrome-win64/chrome.exe'

In [4]:
service = Service('C:/chromedriver/chromedriver-win64/chromedriver.exe')

In [5]:
chrome_options.add_argument('--start-maximized')

In [6]:
# link to trending medium stories
url = 'https://blog.medium.com/trending'

In [7]:
name_of_publisher = []
title_of_article = []
_date_published = []
time_published = []
link_to_article = []
_number_of_responses = []
_number_of_claps = []

In [8]:
def return_date_time(timestamp):
    timestamp = str(timestamp)

    dt = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))

    date = dt.date()
    time = dt.time()
    return [date, time]

In [9]:
# initialize the driver
driver = webdriver.Chrome(service=service,options=chrome_options)

try:
    driver.get(url)
    driver.implicitly_wait(2)

    top_level_article_div = driver.find_element(By.CLASS_NAME, 'js-postListHandle')

    previous_height = 0
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    
        # Get the new scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Check if the scroll height has not increased (indicating the end)
        if new_height == previous_height:
            print("Reached the end of the page.")
            break
        
        # Update the previous height for the next loop iteration
        previous_height = new_height
    
    top_level_for_each_article = top_level_article_div.find_elements(By.CSS_SELECTOR, 
                                                                     "div.postArticle.postArticle--short.js-postArticle.js-trackPostPresentation")
        
    number_of_articles = len(top_level_for_each_article)
        
        # if number_of_articles > 200:
        #     print(f'{number_of_articles}')
        #     break
    
    for i, article in enumerate(top_level_for_each_article):
        name = article.find_element(By.CSS_SELECTOR,
                                    '[class*="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor"]').text
        # print(f'1 {name}')
        try:
            title = article.find_element(By.CSS_SELECTOR, '[class*="graf graf--h2 "]').text
        except Exception:
            try:
                title = article.find_element(By.CSS_SELECTOR, '[class*="graf graf--h3 "]').text
            except Exception:
                title = article.find_element(By.CSS_SELECTOR, '[class*="graf graf--h4 "]').text
    
        # print(f'2 {title}')
        date_published = article.find_element(By.CSS_SELECTOR, "a.link.link--darken time")
        date_published = date_published.get_attribute("datetime")
        # print(f'3 {date_published}')
        link = article.find_element(By.CSS_SELECTOR, 'a.button.button--smaller.button--chromeless.u-baseColor--buttonNormal')
        link = link.get_attribute('href')
        # print(f'4 {link}')
        number_of_responses = article.find_element(By.CSS_SELECTOR, 'div.buttonSet.u-floatRight').text
        number_of_responses = number_of_responses.split(' ')[0]
        # print(f'5 {number_of_responses}')
        number_of_claps = article.find_element(By.CSS_SELECTOR, 
                                               'button.button.button--chromeless.u-baseColor--buttonNormal.js-multirecommendCountButton.u-disablePointerEvents').text
        # print(f'6 {number_of_claps}')

        print(i)
        print(f'Published by: {name}')
        print(f'Title: {title}')
        print(f'Published Date: {date_published}')
        print(f'Link: {link}')
        print(f'Response: {number_of_responses}')
        print(f'Number of claps: {number_of_claps}\n')

        name_of_publisher.append(name)
        title_of_article.append(title)
        dt = return_date_time(date_published)
        _date_published.append(dt[0])
        time_published.append(dt[1])
        link_to_article.append(link)
        _number_of_responses.append(number_of_responses)
        _number_of_claps.append(number_of_claps)
        
except Exception as e:
    print(f'{e}')
finally:
    driver.quit()

Reached the end of the page.
0
Published by: The Medium Newsletter
Title: “Work with people who respect you and who you respect. No exceptions.”
Published Date: 2024-11-15T13:31:52.123Z
Link: https://blog.medium.com/work-with-people-who-respect-you-and-who-you-respect-no-exceptions-1485dd0f6899?source=---------0
Response: 41
Number of claps: 1.8K

1
Published by: Scott Lamb
Title: How we’re approaching AI-generated writing on Medium
Published Date: 2023-01-26T21:42:05.188Z
Link: https://blog.medium.com/how-were-approaching-ai-generated-writing-on-medium-16ee8cb3bc89?source=---------1
Response: 886
Number of claps: 23K

2
Published by: The Medium Newsletter
Title: How to make imperfect decisions
Published Date: 2024-11-25T13:32:06.977Z
Link: https://blog.medium.com/how-to-make-imperfect-decisions-4ada827aebfe?source=---------2
Response: 35
Number of claps: 1.6K

3
Published by: Zulie @ Medium
Title: The 10-year-old Medium story that turned into a book deal
Published Date: 2024-10-01T17:

In [10]:
number_of_articles

114

### Create a CSV of Trending Stories with pandas

In [11]:
pd.set_option('display.max_rows', None) # print all rows

In [12]:
data_to_csv = pd.DataFrame({'name_of_publisher':name_of_publisher, 
                            'title_of_article':title_of_article,
                            'date_published': _date_published,
                            'time_published': time_published,
                            'link_to_article': link_to_article, 
                            'number_of_responses': _number_of_responses,
                            'number_of_claps': _number_of_claps})

In [13]:
data_to_csv

,name_of_publisher,title_of_article,date_published,time_published,link_to_article,number_of_responses,number_of_claps
0,The Medium Newsletter,“Work with people who respect you and who you ...,2024-11-15,13:31:52.123000,https://blog.medium.com/work-with-people-who-r...,41,1.8K
1,Scott Lamb,How we’re approaching AI-generated writing on ...,2023-01-26,21:42:05.188000,https://blog.medium.com/how-were-approaching-a...,886,23K
2,The Medium Newsletter,How to make imperfect decisions,2024-11-25,13:32:06.977000,https://blog.medium.com/how-to-make-imperfect-...,35,1.6K
3,Zulie @ Medium,The 10-year-old Medium story that turned into ...,2024-10-01,17:21:10.444000,https://blog.medium.com/the-10-year-old-medium...,193,11.4K
4,The Medium Newsletter,A former member of the White House transition ...,2024-11-06,15:31:56.106000,https://blog.medium.com/a-former-member-of-the...,17,500
5,The Medium Newsletter,The changing “keys” to the White House,2024-11-14,13:32:27.516000,https://blog.medium.com/the-changing-keys-to-t...,11,359
6,The Medium Newsletter,Dispatches from a post-election classroom,2024-11-12,13:32:00.893000,https://blog.medium.com/dispatches-from-a-post...,15,523
7,The Medium Newsletter,“One must imagine Sisyphus happy”,2024-11-08,13:32:17.408000,https://blog.medium.com/one-must-imagine-sisyp...,10,464
8,Medium,Tips and Tricks for Medium Writers,2017-10-02,23:08:00,https://blog.medium.com/tips-and-tricks-for-me...,445,51K
9,Zulie @ Medium,What you can do with Medium,2024-05-30,13:05:06.307000,https://blog.medium.com/what-you-can-do-with-m...,130,5.4K


### Export Scraped  Trending Stories Details to CSV

In [14]:
data_to_csv.to_csv('trending_medium-stories.csv', encoding='utf-8-sig', index=False)

In [15]:
data = pd.read_csv('trending_medium-stories.csv')

In [16]:
data.head()

,name_of_publisher,title_of_article,date_published,time_published,link_to_article,number_of_responses,number_of_claps
0,The Medium Newsletter,“Work with people who respect you and who you ...,2024-11-15,13:31:52.123000,https://blog.medium.com/work-with-people-who-r...,41.0,1.8K
1,Scott Lamb,How we’re approaching AI-generated writing on ...,2023-01-26,21:42:05.188000,https://blog.medium.com/how-were-approaching-a...,886.0,23K
2,The Medium Newsletter,How to make imperfect decisions,2024-11-25,13:32:06.977000,https://blog.medium.com/how-to-make-imperfect-...,35.0,1.6K
3,Zulie @ Medium,The 10-year-old Medium story that turned into ...,2024-10-01,17:21:10.444000,https://blog.medium.com/the-10-year-old-medium...,193.0,11.4K
4,The Medium Newsletter,A former member of the White House transition ...,2024-11-06,15:31:56.106000,https://blog.medium.com/a-former-member-of-the...,17.0,500
